In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import  ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_length=20,
    max_token_limit=100,
    return_messages=True,

)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "Titanic",
        "answer": """
       🚢💑🧊
        """,
    },
    {
        "movie": "Inception",
        "answer": """
       🧠🌀🕰️
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]



example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert who explains the movie with three emojis."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "What do you wnant to express about {movie}?"),
    ]
)

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)



In [10]:
invoke_chain("Inception")

🧠🌀🕰️content='🧠🌀🕰️'


In [11]:
invoke_chain("Jurassic Park")

🦖🌴🔬content='🦖🌴🔬'


In [12]:
invoke_chain("Frozen") 

❄️👸⛄content='❄️👸⛄'


In [17]:
invoke_chain("Jaws")

🦈🏖️🚤The human mentions "Inception" and the AI responds with emojis related to the movie. When the human brings up "Jurassic Park," the AI responds with emojis related to dinosaurs and science. Lastly, when the human mentions "Frozen," the AI responds with snowflake, princess, and snowman emojis.content='🦈🏖️🚤'


In [16]:
invoke_chain("Star Wars") 

🌌⚔️🤖The human mentions "Inception" and the AI responds with emojis related to the movie. The human then brings up "Jurassic Park," and the AI responds with emojis related to dinosaurs and science. When the human mentions "Frozen," the AI responds with snowflake, princess, and snowman emojis.content='🌌⚔️🤖'


In [18]:
invoke_chain("King Kong")


🦍🏢🌆The human mentions "Inception" and the AI responds with emojis related to the movie. When the human brings up "Jurassic Park," the AI responds with emojis related to dinosaurs and science. Lastly, when the human mentions "Frozen," the AI responds with ❄️👸⛄ emojis.content='🦍🏢🌆'
